# Lab: Value Function Iteration for Two Models
Compact notebook for a lab session on value function iteration (VFI) applied to:
1. McCall job-search model (with separation).
2. Deterministic optimal growth (Cobb–Douglas).

Prereqs: Python 3.10+, numpy, matplotlib. Run cells top to bottom.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

## 1. McCall job-search model (with separation)
State: unemployment or employment at wage \(w\).  
Action: accept an offer \(w\) (if unemployed) or continue (if employed).  
Shock: i.i.d. wage offer \(W\) with discrete distribution on a grid \(\{w_i, p_i\}\).  
Flow payoff: unemployed \(b\); employed at \(w\) gets \(w\).  
Separation prob.: \(\lambda\in[0,1)\).

Bellman (unemployed):
\[
V_U \;=\; b \;+\; \beta\,\mathbb{E}\big[\,\max\{V_U,\; V_E(W)\}\,\big].
\]
Bellman (employed at \(w_i\)):
\[
V_E(w_i) \;=\; w_i \;+\; \beta\big[(1-\lambda)\,V_E(w_i)\;+\;\lambda\,V_U\big].
\]
Acceptance: accept iff \(V_E(w)\ge V_U\). Reservation wage is the smallest \(w\) with this property.

In [ ]:
# Parameters
beta   = 0.95        # discount
lam    = 0.10        # separation probability
b      = 1.0         # unemployment payoff
w_grid = np.linspace(0.5, 5.0, 80)  # wage offer grid
# Discrete lognormal-like weights normalized
mu, sigma = 1.2, 0.5
pdf = (1/(w_grid*sigma*np.sqrt(2*np.pi))) * np.exp(-(np.log(w_grid)-mu)**2/(2*sigma**2))
p = pdf / pdf.sum()

# Initialization
VU = 0.0
VE = np.zeros_like(w_grid)

def bellman_step(VU, VE):
    VE_new = w_grid + beta*((1.0 - lam)*VE + lam*VU)  # employed update
    cont = np.maximum(VU, VE_new)                     # accept-or-reject
    VU_new = b + beta*np.dot(p, cont)                 # unemployed update
    return VU_new, VE_new

# Iterate
tol = 1e-8
for it in range(10000):
    VU_new, VE_new = bellman_step(VU, VE)
    diff = max(abs(VU_new - VU), np.max(abs(VE_new - VE)))
    VU, VE = VU_new, VE_new
    if diff < tol:
        break

idx = np.argmax(VE >= VU)  # first index meeting VE>=VU
w_res = w_grid[idx] if VE[idx] >= VU else np.nan

print(f"Converged in {it+1} iterations")
print(f"V_U = {VU:.6f}")
print(f"Reservation wage w* = {w_res:.4f}")

In [ ]:
# Plot for McCall
fig, ax = plt.subplots(figsize=(5,3.2))
ax.plot(w_grid, VE, label="V_E(w)")
ax.axhline(VU, linestyle='--', label="V_U")
if np.isfinite(w_res):
    ax.axvline(w_res, linestyle=':', label="w*")
ax.set_xlabel("w")
ax.set_ylabel("Value")
ax.set_title("McCall: employed vs. unemployment value")
ax.legend()
plt.show()

## 2. Deterministic optimal growth (Cobb–Douglas)
Resource: \(y=f(k)=z\,k^{\alpha}\). Capital law: \(k' = f(k) + (1-\delta)k - c\).  
Control \(k'\) with \(c\ge 0\).

Bellman:
\[
V(k) \;=\; \max_{k'} \left\{ u\!\big(f(k)+(1-\delta)k - k'\big) \;+\; \beta\,V(k') \right\},
\]
with \(u(c)=\frac{c^{1-\sigma}}{1-\sigma}\), \(\sigma\neq 1\).

In [ ]:
# Parameters
beta   = 0.95
alpha  = 0.35
delta  = 0.08
z      = 1.0
sigma  = 2.0

# Grid
k_min, k_max, k_size = 1e-3, 10.0, 600
k_grid = np.linspace(k_min, k_max, k_size)
f = lambda k: z * k**alpha
y_grid = f(k_grid) + (1.0 - delta)*k_grid

V = np.zeros_like(k_grid)
g = np.copy(k_grid)

tol = 1e-7
for it in range(2000):
    V_new = np.empty_like(V)
    g_new = np.empty_like(g)
    for i, k in enumerate(k_grid):
        y = y_grid[i]
        jmax = np.searchsorted(k_grid, y)  # enforce c>=0
        if jmax == 0:
            V_new[i] = -1e9
            g_new[i] = k_grid[0]
            continue
        k_choices = k_grid[:jmax]
        c_choices = y - k_choices
        # CRRA utility (vectorized)
        obj = (c_choices**(1.0 - sigma))/(1.0 - sigma) + beta * V[:jmax]
        j_star = np.argmax(obj)
        V_new[i] = obj[j_star]
        g_new[i] = k_choices[j_star]
    diff = np.max(np.abs(V_new - V))
    V, g = V_new, g_new
    if diff < tol:
        break

print(f"Converged in {it+1} iterations")

In [ ]:
# Plots
fig, ax = plt.subplots(1, 2, figsize=(9,3.2))

ax[0].plot(k_grid, V)
ax[0].set_title("Value function V(k)")
ax[0].set_xlabel("k"); ax[0].set_ylabel("V(k)")

ax[1].plot(k_grid, g, label="k'(k)")
ax[1].plot(k_grid, k_grid, linestyle='--', label="45°")
ax[1].set_title("Policy function")
ax[1].set_xlabel("k"); ax[1].set_ylabel("k'"); ax[1].legend()

plt.tight_layout()
plt.show()